# 病床姿勢監測 - 資料準備與對齊（Version 3 - 修正時間戳）

**專案**: 熱成像超解析度 + 姿勢辨識  
**作者**: rochi190  
**日期**: 2025-11-04  
**版本**: v3 - 修正 Thermal 時間戳解析（手動加上毫秒偏移）

## 修正重點
- ✅ **Thermal 時間戳只有秒級精度**，手動加上毫秒偏移（125ms/幀）
- ✅ 每個 Thermal 幀配對到不同的 RGB 幀（不再重複）
- ✅ RGB 幀間隔約 3 幀（符合 8fps vs 24.67fps 的比例）

## 功能
1. 讀取 output.txt 取得相機開始時間
2. 載入熱成像 log 檔案（多核加速 + **修正時間戳**）
3. 分析 RGB 影片資訊
4. **Adaptive 配對策略**（處理不固定 FPS 和資料遺漏）
5. 翻轉檢測與修復
6. 匯出 RGB 圖片供 LabelMe 標註（多核加速）
7. 儲存對齊後的 Thermal 資料

## 輸出
- `output/labelme_project/` - LabelMe 標註專案
- `output/aligned_dataset/` - 對齊後的訓練資料
- `output/adaptive_pairing/` - 配對分析結果

---
## Step 0: 環境設定與套件匯入

In [ ]:
import os
import sys
import json
import re
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime, timedelta
from typing import List, Dict, Tuple, Optional, Any
from tqdm import tqdm
from dataclasses import dataclass, field
from concurrent.futures import ProcessPoolExecutor
from collections import defaultdict, Counter
import multiprocessing as mp
import warnings
warnings.filterwarnings('ignore')

# Matplotlib 設定
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.dpi'] = 100

# Seaborn 樣式
sns.set_style('whitegrid')

print("="*70)
print("病床姿勢監測系統 - 資料準備模組 v3（修正時間戳）")
print("="*70)
print(f"執行時間: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"使用者: rochi190")
print(f"Python 版本: {sys.version}")
print(f"NumPy 版本: {np.__version__}")
print(f"OpenCV 版本: {cv2.__version__}")
print(f"CPU 核心數: {mp.cpu_count()}")
print("="*70)

---
## Step 1: 路徑與參數配置

In [ ]:
# ========================================
# 多核處理配置
# ========================================
NUM_WORKERS = max(1, mp.cpu_count() - 1)
CHUNK_SIZE = 100

print(f"🚀 多核處理配置:")
print(f"  使用 Workers: {NUM_WORKERS}")
print(f"  批次大小: {CHUNK_SIZE}")

# ========================================
# 路徑配置
# ========================================
BASE_DIR = Path(os.getcwd())
DATA_DIR = BASE_DIR / 'data'
THERMAL_DIR = DATA_DIR / 'thermal'
CAMERA_DIR = DATA_DIR / 'camera'
OUTPUT_TXT = CAMERA_DIR / 'output.txt'
AVI_FILE = CAMERA_DIR / 'output.avi'

# 輸出目錄
OUTPUT_DIR = BASE_DIR / 'output1'
ADAPTIVE_DIR = OUTPUT_DIR / 'adaptive_pairing'
LABELME_DIR = OUTPUT_DIR / 'labelme_project'
ALIGNED_DIR = OUTPUT_DIR / 'aligned_dataset'
DIAGNOSTIC_DIR = OUTPUT_DIR / 'diagnostics'

for directory in [ADAPTIVE_DIR, LABELME_DIR, ALIGNED_DIR, DIAGNOSTIC_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

# ========================================
# 熱成像參數
# ========================================
THERMAL_RESOLUTION = (32, 24)  # (寬度, 高度)
THERMAL_WIDTH, THERMAL_HEIGHT = THERMAL_RESOLUTION

# ========================================
# Adaptive 配對參數
# ========================================
TARGET_FPS = 8  # 目標 FPS
NORMAL_FRAMES_PER_SEC = 8  # 正常每秒幀數
FRAME_INTERVAL_MS = 1000 / TARGET_FPS  # 125ms

# 翻轉設定（執行 Step 8 後調整）
FLIP_THERMAL = True
FLIP_RGB = False

# ========================================
# 顯示配置
# ========================================
print(f"\n📁 路徑檢查:")
print(f"  Thermal 目錄: {'✓' if THERMAL_DIR.exists() else '✗'} {THERMAL_DIR}")
print(f"  Camera 目錄: {'✓' if CAMERA_DIR.exists() else '✗'} {CAMERA_DIR}")
print(f"  output.txt: {'✓' if OUTPUT_TXT.exists() else '✗'} {OUTPUT_TXT}")
print(f"  AVI 檔案: {'✓' if AVI_FILE.exists() else '✗'} {AVI_FILE}")

print(f"\n⚙️ 配對參數:")
print(f"  目標 FPS: {TARGET_FPS}")
print(f"  幀間隔: {FRAME_INTERVAL_MS:.1f}ms")

print(f"\n🔄 翻轉設定:")
print(f"  翻轉 Thermal: {FLIP_THERMAL}")
print(f"  翻轉 RGB: {FLIP_RGB}")

---
## Step 2: 解析 output.txt（相機開始時間）

In [ ]:
def parse_output_txt(txt_path: Path) -> Optional[datetime]:
    """
    解析 output.txt 取得相機開始時間
    """
    if not txt_path.exists():
        print(f"❌ {txt_path} 不存在")
        return None
    
    try:
        with open(txt_path, 'r', encoding='utf-8') as f:
            content = f.read()
    except:
        with open(txt_path, 'r', encoding='big5') as f:
            content = f.read()
    
    print(f"\n{'='*70}")
    print(f"解析 output.txt")
    print(f"{'='*70}")
    print(f"檔案大小: {len(content)} bytes")
    print(f"\n前 500 字元:\n{content[:500]}")
    print(f"{'='*70}")
    
    # 時間格式模式
    patterns = [
        (r'(\d{4}-\d{2}-\d{2}\s+\d{2}:\d{2}:\d{2})', '%Y-%m-%d %H:%M:%S'),
        (r'(\d{4}/\d{2}/\d{2}\s+\d{2}:\d{2}:\d{2})', '%Y/%m/%d %H:%M:%S'),
    ]
    
    for pattern, fmt in patterns:
        match = re.search(pattern, content)
        if match:
            try:
                camera_start = datetime.strptime(match.group(1), fmt)
                print(f"\n✅ 找到相機開始時間: {camera_start}")
                return camera_start
            except:
                continue
    
    print(f"\n❌ 無法解析時間")
    return None

# 執行解析
camera_start_time = parse_output_txt(OUTPUT_TXT)

if camera_start_time is None:
    raise RuntimeError("❌ 無法解析 output.txt，請檢查檔案格式")

print(f"\n📌 相機開始時間: {camera_start_time}")

---
## Step 3: 載入熱成像資料（修正版：手動加上毫秒偏移）

In [ ]:
def parse_thermal_file(file_path: Path) -> List[Dict]:
    """
    解析單一熱成像 log 檔案（修正版）
    
    修正：
    - time 格式: [年, 月, 日, 時, 分, 秒, 星期, 年中天數, 夏令時]
    - 沒有毫秒資訊，手動根據順序加上偏移
    - 假設 8fps，每幀間隔 125ms
    """
    data = []
    
    # 用於追蹤每一秒內的幀數
    second_frame_counts = {}
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                
                try:
                    data_dict = json.loads(line)
                    time_parts = data_dict.get('time', [])
                    
                    if len(time_parts) < 6:
                        continue
                    
                    year, month, day, hour, minute, second = time_parts[:6]
                    
                    # 建立秒級別的 key
                    second_key = (year, month, day, hour, minute, second)
                    
                    # 計算這一秒內的第幾幀
                    if second_key not in second_frame_counts:
                        second_frame_counts[second_key] = 0
                    else:
                        second_frame_counts[second_key] += 1
                    
                    frame_in_second = second_frame_counts[second_key]
                    
                    # 🔧 手動計算毫秒偏移
                    millisecond = int(frame_in_second * FRAME_INTERVAL_MS)
                    
                    # 建立時間戳
                    timestamp = datetime(year, month, day, hour, minute, second, 
                                        min(millisecond * 1000, 999999))
                    
                    message = data_dict.get('message', [])
                    if len(message) != THERMAL_WIDTH * THERMAL_HEIGHT:
                        continue
                    
                    thermal_image = np.array(message, dtype=np.uint8).reshape(
                        THERMAL_HEIGHT, THERMAL_WIDTH
                    )
                    
                    data.append({
                        'timestamp': timestamp,
                        'image': thermal_image
                    })
                    
                except:
                    continue
    
    except Exception as e:
        print(f"❌ 載入失敗 {file_path.name}: {e}")
        return []
    
    return data

# 載入所有 thermal log
log_files = sorted(THERMAL_DIR.glob('log*.txt'))

if not log_files:
    raise FileNotFoundError(f"❌ 未找到 log 檔案於 {THERMAL_DIR}")

print(f"\n{'='*70}")
print(f"載入熱成像資料（多核: {NUM_WORKERS} workers）")
print(f"{'='*70}")
print(f"找到 {len(log_files)} 個 log 檔案")
print(f"🔧 修正：手動加上毫秒偏移（{FRAME_INTERVAL_MS:.1f}ms/幀）")

therm_data = []
with ProcessPoolExecutor(max_workers=NUM_WORKERS) as executor:
    results = list(tqdm(
        executor.map(parse_thermal_file, log_files),
        total=len(log_files),
        desc="載入 Thermal"
    ))

for file_data in results:
    therm_data.extend(file_data)

# 排序
therm_data.sort(key=lambda x: x['timestamp'])

print(f"\n✅ 載入完成")
print(f"  總幀數: {len(therm_data):,}")
if therm_data:
    thermal_start = therm_data[0]['timestamp']
    thermal_end = therm_data[-1]['timestamp']
    duration = (thermal_end - thermal_start).total_seconds()
    print(f"  時間範圍: {thermal_start} ~ {thermal_end}")
    print(f"  時長: {duration:.2f} 秒")
    print(f"  平均 FPS: {len(therm_data) / duration:.2f}")
    
    # 檢查時間間隔
    time_diffs = []
    for i in range(min(100, len(therm_data) - 1)):
        dt = (therm_data[i+1]['timestamp'] - therm_data[i]['timestamp']).total_seconds() * 1000
        time_diffs.append(dt)
    
    print(f"\n  時間間隔統計（前 100 幀）:")
    print(f"    平均: {np.mean(time_diffs):.1f}ms")
    print(f"    中位數: {np.median(time_diffs):.1f}ms")
    print(f"    預期: {FRAME_INTERVAL_MS:.1f}ms")

print("="*70)

---
## Step 4: 分析 RGB 影片

In [ ]:
# 分析 AVI 影片
cap = cv2.VideoCapture(str(AVI_FILE))

if not cap.isOpened():
    raise RuntimeError(f"❌ 無法開啟影片 {AVI_FILE}")

fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
duration = frame_count / fps if fps > 0 else 0

cap.release()

print(f"\n{'='*70}")
print(f"RGB 影片資訊")
print(f"{'='*70}")
print(f"  FPS: {fps:.2f}")
print(f"  總幀數: {frame_count:,}")
print(f"  解析度: {width} × {height}")
print(f"  時長: {duration:.2f} 秒")
print(f"  開始時間: {camera_start_time}")
print(f"  結束時間: {camera_start_time + timedelta(seconds=duration)}")
print(f"\n  預期 Thermal-RGB 幀間隔:")
print(f"    Thermal 間隔: {FRAME_INTERVAL_MS:.1f}ms")
print(f"    RGB 間隔: {1000/fps:.1f}ms")
print(f"    每個 Thermal 對應 RGB 幀數: {FRAME_INTERVAL_MS / (1000/fps):.1f}")
print("="*70)

step 4.5

In [ ]:
def analyze_thermal_distribution(thermal_data: List[Dict], camera_start_time: datetime) -> Dict:
    """
    分析熱成像資料的時間分佈
    將幀按秒分組
    """
    sections = {}
    
    for frame in thermal_data:
        offset = (frame['timestamp'] - camera_start_time).total_seconds()
        
        if offset < 0:
            continue
        
        second = int(offset)
        if second not in sections:
            sections[second] = []
        sections[second].append(frame)
    
    print(f"\n{'='*70}")
    print(f"熱成像時間分佈分析")
    print(f"{'='*70}")
    
    section_counts = {sec: len(frames) for sec, frames in sections.items()}
    counts = list(section_counts.values())
    
    print(f"\n總秒數: {len(sections)}")
    print(f"最小幀數/秒: {min(counts)}")
    print(f"最大幀數/秒: {max(counts)}")
    print(f"平均幀數/秒: {np.mean(counts):.1f}")
    print(f"標準差: {np.std(counts):.1f}")
    
    # 分類
    normal_seconds = [sec for sec, count in section_counts.items() if count == NORMAL_FRAMES_PER_SEC]
    dropout_seconds = [sec for sec, count in section_counts.items() if count < NORMAL_FRAMES_PER_SEC]
    burst_seconds = {sec: count for sec, count in section_counts.items() if count > NORMAL_FRAMES_PER_SEC}
    
    print(f"\n正常 ({NORMAL_FRAMES_PER_SEC} 張/秒): {len(normal_seconds)} 秒")
    print(f"Dropout (<{NORMAL_FRAMES_PER_SEC} 張/秒): {len(dropout_seconds)} 秒")
    print(f"Burst (>{NORMAL_FRAMES_PER_SEC} 張/秒): {len(burst_seconds)} 秒")
    
    if burst_seconds:
        print(f"\nBurst 範例（前 5 秒）:")
        for sec, count in sorted(burst_seconds.items(), key=lambda x: x[1], reverse=True)[:5]:
            print(f"  第 {sec} 秒: {count} 張 (多 {count - NORMAL_FRAMES_PER_SEC} 張)")
    
    print("="*70)
    
    return sections

# 執行分析
sections = analyze_thermal_distribution(therm_data, camera_start_time)

---
## Step 5: 簡化配對（直接按時間戳配對）

In [ ]:
def adaptive_thermal_rgb_pairing_v6(
    thermal_data: List[Dict],
    camera_start_time: datetime,
    fps: float,
    frame_count: int,
    sections: Dict
) -> Tuple[List[Dict], List[Dict]]:
    """
    Adaptive 配對策略 v6（最終優化版）
    
    優化：
    1. 多 1-2 張的輕微 burst 直接丟棄多餘幀（不重新分配）
    2. 只處理嚴重 burst（≥ 10 張/秒，即多 ≥ 2 張）
    3. 避免產生 interval 0 的情況
    """
    print(f"\n{'='*70}")
    print(f"Adaptive Thermal-RGB 配對 v6（最終優化版）")
    print(f"{'='*70}")
    
    # 1. 找出完全缺失的秒
    if not sections:
        print("❌ 無資料")
        return [], []
    
    min_sec = min(sections.keys())
    max_sec = max(sections.keys())
    
    all_seconds = set(range(min_sec, max_sec + 1))
    existing_seconds = set(sections.keys())
    missing_seconds = sorted(all_seconds - existing_seconds)
    
    print(f"\n1. 缺失檢測:")
    print(f"   時間範圍: 第 {min_sec} ~ {max_sec} 秒")
    print(f"   完全缺失的秒: {len(missing_seconds)}")
    
    # 為缺失的秒建立空的 section
    for sec in missing_seconds:
        sections[sec] = []
    
    # 2. 分類並處理 burst
    serious_bursts = {}  # ≥ 10 張/秒（多 ≥ 2 張）
    minor_bursts = []    # 9 張/秒（多 1 張）
    severe_dropout = []  # ≤ 3 張/秒
    
    SEVERE_DROPOUT_THRESHOLD = 3
    SERIOUS_BURST_THRESHOLD = 10  # ≥ 10 張才重新分配
    REDISTRIBUTION_WINDOW = 15
    
    print(f"\n2. 分類與預處理:")
    
    for sec in sorted(sections.keys()):
        count = len(sections[sec])
        
        if count >= SERIOUS_BURST_THRESHOLD:
            # 嚴重 burst：需要重新分配
            serious_bursts[sec] = count
        elif count == NORMAL_FRAMES_PER_SEC + 1:
            # 輕微 burst（多 1 張）：直接丟棄最後一張
            minor_bursts.append(sec)
            # 🔧 保留前 8 張，丟棄第 9 張
            sections[sec] = sections[sec][:NORMAL_FRAMES_PER_SEC]
        elif count <= SEVERE_DROPOUT_THRESHOLD:
            # 嚴重 dropout
            severe_dropout.append(sec)
    
    print(f"   嚴重 Burst (≥{SERIOUS_BURST_THRESHOLD} 張/秒): {len(serious_bursts)}")
    print(f"   輕微 Burst (9 張/秒): {len(minor_bursts)} ← 直接丟棄多餘幀")
    print(f"   嚴重 Dropout (≤{SEVERE_DROPOUT_THRESHOLD} 張/秒): {len(severe_dropout)}")
    
    if serious_bursts:
        print(f"\n   嚴重 Burst 列表:")
        for sec in sorted(serious_bursts.keys(), key=lambda x: serious_bursts[x], reverse=True)[:5]:
            count = serious_bursts[sec]
            print(f"      第 {sec} 秒: {count} 張 (多 {count - NORMAL_FRAMES_PER_SEC} 張)")
    
    # 3. 智能重新分配（只處理嚴重 burst）
    redistribution_log = []
    total_redistributed = 0
    
    print(f"\n3. 智能重新分配:")
    
    for sec in sorted(serious_bursts.keys()):
        frames = sections[sec]
        excess_count = len(frames) - NORMAL_FRAMES_PER_SEC
        
        # 找窗口內的嚴重 dropout
        window_start = max(min_sec, sec - REDISTRIBUTION_WINDOW)
        window_dropouts = [s for s in severe_dropout if window_start <= s < sec]
        
        if not window_dropouts:
            print(f"   第 {sec} 秒的 burst ({excess_count} 幀) → 無可分配目標（丟棄）")
            # 保留後 8 張
            sections[sec] = frames[-NORMAL_FRAMES_PER_SEC:]
            continue
        
        print(f"   第 {sec} 秒的 burst ({excess_count} 幀) → 分配給前面 {len(window_dropouts)} 個嚴重 dropout:")
        
        # 取出多餘的幀（前面的）
        excess_frames = frames[:excess_count]
        normal_frames = frames[excess_count:]
        
        sections[sec] = normal_frames
        
        # 分配給 dropout
        frame_idx = 0
        for dropout_sec in sorted(window_dropouts):
            if frame_idx >= len(excess_frames):
                break
            
            current_count = len(sections[dropout_sec])
            needed = NORMAL_FRAMES_PER_SEC - current_count
            available = min(needed, len(excess_frames) - frame_idx)
            
            for i in range(available):
                frame = excess_frames[frame_idx]
                
                # 修正時間戳
                frame_in_sec = len(sections[dropout_sec])
                new_millisecond = int(frame_in_sec * FRAME_INTERVAL_MS)
                
                base_time = camera_start_time + timedelta(seconds=dropout_sec)
                new_timestamp = base_time + timedelta(milliseconds=new_millisecond)
                
                frame['timestamp'] = new_timestamp
                sections[dropout_sec].append(frame)
                
                redistribution_log.append({
                    'from_sec': sec,
                    'to_sec': dropout_sec,
                    'frame': frame
                })
                
                frame_idx += 1
            
            if available > 0:
                print(f"      → 第 {dropout_sec} 秒補了 {available} 幀")
        
        total_redistributed += frame_idx
        
        if frame_idx < len(excess_frames):
            discarded = len(excess_frames) - frame_idx
            print(f"      ⚠️ 仍有 {discarded} 幀未分配（丟棄）")
    
    print(f"\n   總共重新分配: {total_redistributed} 幀")
    print(f"   輕微 burst 丟棄: {len(minor_bursts)} 幀")
    
    # 4. 按時間戳排序
    for sec in sections.keys():
        sections[sec].sort(key=lambda x: x['timestamp'])
    
    # 5. 配對 Thermal 和 RGB
    pairs = []
    rgb_end_time = camera_start_time + timedelta(seconds=frame_count / fps)
    
    for sec in sorted(sections.keys()):
        if not sections[sec]:
            continue
        
        for thermal_frame in sections[sec]:
            thermal_ts = thermal_frame['timestamp']
            
            if thermal_ts < camera_start_time or thermal_ts > rgb_end_time:
                continue
            
            offset = (thermal_ts - camera_start_time).total_seconds()
            rgb_frame_idx = int(round(offset * fps))
            rgb_frame_idx = max(0, min(rgb_frame_idx, frame_count - 1))
            
            actual_time = rgb_frame_idx / fps
            rgb_error = abs(actual_time - offset)
            
            pairs.append({
                'pair_id': f'pair_{len(pairs):05d}',
                'thermal': thermal_frame,
                'rgb_frame_idx': rgb_frame_idx,
                'timestamp': thermal_ts,
                'section': sec,
                'rgb_error_ms': rgb_error * 1000
            })
    
    print(f"\n4. 最終配對結果:")
    print(f"   總配對數: {len(pairs):,}")
    print(f"   成功率: {len(pairs) / len(thermal_data) * 100:.1f}%")
    
    if pairs:
        rgb_errors = [p['rgb_error_ms'] for p in pairs]
        print(f"\n   RGB 誤差: 平均 {np.mean(rgb_errors):.2f}ms")
        
        rgb_indices = [p['rgb_frame_idx'] for p in pairs]
        intervals = [rgb_indices[i+1] - rgb_indices[i] for i in range(len(rgb_indices) - 1)]
        interval_counts = Counter(intervals)
        
        print(f"\n   RGB 幀間隔分佈:")
        normal_intervals = {k: v for k, v in interval_counts.items() if 0 <= k <= 10}
        for interval in sorted(normal_intervals.keys()):
            count = normal_intervals[interval]
            percentage = count / len(intervals) * 100
            print(f"   間隔 {interval:3d} 幀: {count:5d} 次 ({percentage:5.1f}%)")
        
        abnormal = {k: v for k, v in interval_counts.items() if k < 0 or k > 10}
        if abnormal:
            print(f"\n   ⚠️ 異常間隔:")
            print(f"   負值: {sum(v for k, v in abnormal.items() if k < 0)} 次")
            print(f"   大跳躍: {sum(v for k, v in abnormal.items() if k > 10)} 次")
        else:
            print(f"\n   ✓ 無異常間隔")
    
    print("="*70)
    
    return pairs, redistribution_log

# 執行配對
pairs_adaptive, redistribution_log = adaptive_thermal_rgb_pairing_v6(
    therm_data, camera_start_time, fps, frame_count, sections
)

print(f"\n📌 配對結果: {len(pairs_adaptive):,} 對")

---
## Step 6: 驗證配對結果

In [ ]:
# 檢查配對結果
print(f"\n{'='*70}")
print(f"配對結果驗證")
print(f"{'='*70}")

print(f"\n前 20 個配對:")
for i, pair in enumerate(pairs_adaptive[:20]):
    thermal_time = pair['timestamp'].strftime('%H:%M:%S.%f')[:-3]
    rgb_idx = pair['rgb_frame_idx']
    error = pair['rgb_error_ms']
    
    # 計算與前一幀的間隔
    if i > 0:
        time_diff = (pair['timestamp'] - pairs_adaptive[i-1]['timestamp']).total_seconds() * 1000
        rgb_diff = rgb_idx - pairs_adaptive[i-1]['rgb_frame_idx']
        print(f"{i:2d}. {pair['pair_id']}: Thermal {thermal_time} → RGB 幀 {rgb_idx:5d} (誤差 {error:4.1f}ms, 時間間隔 {time_diff:6.1f}ms, RGB間隔 {rgb_diff} 幀)")
    else:
        print(f"{i:2d}. {pair['pair_id']}: Thermal {thermal_time} → RGB 幀 {rgb_idx:5d} (誤差 {error:4.1f}ms)")

print("="*70)

---
## Step 7: 視覺化分析

In [ ]:
# 視覺化配對結果
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. RGB 誤差分布
rgb_errors = [p['rgb_error_ms'] for p in pairs_adaptive]
axes[0, 0].hist(rgb_errors, bins=50, edgecolor='black', alpha=0.7, color='#51cf66')
axes[0, 0].axvline(np.mean(rgb_errors), color='red', linestyle='--', linewidth=2, 
                   label=f'Mean: {np.mean(rgb_errors):.2f}ms')
axes[0, 0].set_xlabel('RGB Error (ms)')
axes[0, 0].set_ylabel('Count')
axes[0, 0].set_title('RGB Time Alignment Error Distribution')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. RGB 幀間隔分佈
rgb_indices = [p['rgb_frame_idx'] for p in pairs_adaptive]
intervals = [rgb_indices[i+1] - rgb_indices[i] for i in range(len(rgb_indices) - 1)]

axes[0, 1].hist(intervals, bins=range(0, max(intervals)+2), edgecolor='black', alpha=0.7, color='#74c0fc')
expected_interval = FRAME_INTERVAL_MS / (1000 / fps)
axes[0, 1].axvline(expected_interval, color='red', linestyle='--', linewidth=2,
                   label=f'Expected: {expected_interval:.1f} frames')
axes[0, 1].set_xlabel('RGB Frame Interval')
axes[0, 1].set_ylabel('Count')
axes[0, 1].set_title('RGB Frame Interval Distribution')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. Thermal 時間間隔
time_diffs = []
for i in range(len(pairs_adaptive) - 1):
    dt = (pairs_adaptive[i+1]['timestamp'] - pairs_adaptive[i]['timestamp']).total_seconds() * 1000
    time_diffs.append(dt)

axes[1, 0].hist(time_diffs, bins=50, edgecolor='black', alpha=0.7, color='#ffa94d')
axes[1, 0].axvline(FRAME_INTERVAL_MS, color='red', linestyle='--', linewidth=2,
                   label=f'Expected: {FRAME_INTERVAL_MS:.1f}ms')
axes[1, 0].set_xlabel('Thermal Time Interval (ms)')
axes[1, 0].set_ylabel('Count')
axes[1, 0].set_title('Thermal Frame Time Interval Distribution')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# 4. RGB 幀序列（前 200 對）
sample_size = min(200, len(rgb_indices))
axes[1, 1].plot(range(sample_size), rgb_indices[:sample_size], 'b-', linewidth=1, alpha=0.7)
axes[1, 1].scatter(range(sample_size), rgb_indices[:sample_size], c='blue', s=10, alpha=0.5)
axes[1, 1].set_xlabel('Pair Index')
axes[1, 1].set_ylabel('RGB Frame Index')
axes[1, 1].set_title(f'RGB Frame Sequence (First {sample_size} Pairs)')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
save_path = ADAPTIVE_DIR / 'pairing_analysis_v3.png'
plt.savefig(save_path, dpi=150, bbox_inches='tight')
plt.show()

print(f"\n✅ 視覺化已儲存: {save_path}")

---
## Step 8: 翻轉檢測

In [ ]:
def check_flip_orientation(pairs: List[Dict], avi_path: Path, num_samples: int = 6) -> None:
    """
    檢查是否需要左右翻轉
    """
    import random
    
    if not pairs:
        print("❌ 無配對資料可檢查")
        return
    
    print(f"\n{'='*70}")
    print(f"翻轉方向檢測")
    print(f"{'='*70}")
    
    # 隨機取樣
    sample_size = min(num_samples, len(pairs))
    sample_indices = random.sample(range(len(pairs)), sample_size)
    sample_pairs = [pairs[i] for i in sample_indices]
    
    cap = cv2.VideoCapture(str(avi_path))
    
    # 4 種組合
    fig, axes = plt.subplots(num_samples, 8, figsize=(24, num_samples*3))
    if num_samples == 1:
        axes = axes.reshape(1, -1)
    
    for row_idx, sample_pair in enumerate(sample_pairs):
        thermal = sample_pair['thermal']['image']
        
        # 讀取 RGB
        cap.set(cv2.CAP_PROP_POS_FRAMES, sample_pair['rgb_frame_idx'])
        ret, rgb = cap.read()
        
        if not ret:
            continue
        
        rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
        
        # 4 種組合
        combinations = [
            (thermal, rgb, "原始", "Both Original"),
            (cv2.flip(thermal, 1), rgb, "T翻轉", "Thermal Flipped"),
            (thermal, cv2.flip(rgb, 1), "R翻轉", "RGB Flipped"),
            (cv2.flip(thermal, 1), cv2.flip(rgb, 1), "都翻轉", "Both Flipped")
        ]
        
        for col_idx, (t, r, label, desc) in enumerate(combinations):
            t_up = cv2.resize(t, (r.shape[1], r.shape[0]), interpolation=cv2.INTER_CUBIC)
            
            # Thermal
            axes[row_idx, col_idx*2].imshow(t_up, cmap='hot')
            axes[row_idx, col_idx*2].set_title(f"{label}\nThermal", fontsize=10)
            axes[row_idx, col_idx*2].axis('off')
            
            # RGB
            axes[row_idx, col_idx*2+1].imshow(r)
            axes[row_idx, col_idx*2+1].set_title(f"{label}\nRGB", fontsize=10)
            axes[row_idx, col_idx*2+1].axis('off')
    
    cap.release()
    
    plt.suptitle('翻轉方向檢測 - 請選擇畫面對應最好的組合', fontsize=14, fontweight='bold')
    plt.tight_layout()
    
    save_path = DIAGNOSTIC_DIR / 'flip_comparison.png'
    plt.savefig(save_path, dpi=120, bbox_inches='tight')
    print(f"\n✅ 翻轉比對圖已儲存: {save_path}")
    plt.show()
    
    print(f"\n📝 請檢查圖片，判斷哪種組合正確:")
    print(f"  選項 1 - 原始: FLIP_THERMAL=False, FLIP_RGB=False")
    print(f"  選項 2 - T翻轉: FLIP_THERMAL=True, FLIP_RGB=False")
    print(f"  選項 3 - R翻轉: FLIP_THERMAL=False, FLIP_RGB=True")
    print(f"  選項 4 - 都翻轉: FLIP_THERMAL=True, FLIP_RGB=True")
    print(f"\n確認後請修改 Step 1 的 FLIP_THERMAL 和 FLIP_RGB 參數")
    print("="*70)

# 執行翻轉檢測
check_flip_orientation(pairs_adaptive, AVI_FILE)

---
## Step 9: 儲存配對結果

In [ ]:
# 保存配對結果為 JSON 和 CSV
pairs_export = []
for pair in pairs_adaptive:
    pairs_export.append({
        'pair_id': pair['pair_id'],
        'rgb_frame_idx': pair['rgb_frame_idx'],
        'timestamp': pair['timestamp'].isoformat(),
        'rgb_error_ms': pair['rgb_error_ms'],
        'label': 'thermal',
        'modality': 'thermal-rgb'
    })

# 儲存 JSON
json_path = ADAPTIVE_DIR / 'pairs_mapping_v3.json'
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(pairs_export, f, indent=2, ensure_ascii=False)

# 儲存 CSV
csv_path = ADAPTIVE_DIR / 'pairs_mapping_v3.csv'
df_pairs = pd.DataFrame(pairs_export)
df_pairs.to_csv(csv_path, index=False, encoding='utf-8-sig')

print(f"\n{'='*70}")
print(f"配對結果已儲存")
print(f"{'='*70}")
print(f"  JSON: {json_path}")
print(f"  CSV: {csv_path}")
print(f"  總配對數: {len(pairs_export):,}")
print("="*70)

---
## Step 10: 匯出 RGB 圖片供 LabelMe 標註

In [ ]:
def export_rgb_batch(args: Tuple) -> int:
    """
    批次匯出 RGB 圖片（多核處理用）
    """
    pairs_batch, avi_path, output_dir, flip_rgb = args
    
    cap = cv2.VideoCapture(str(avi_path))
    exported_count = 0
    
    for pair in pairs_batch:
        cap.set(cv2.CAP_PROP_POS_FRAMES, pair['rgb_frame_idx'])
        ret, frame = cap.read()
        
        if not ret:
            continue
        
        if flip_rgb:
            frame = cv2.flip(frame, 1)
        
        # 儲存為 JPG
        filename = f"frame_{pair['pair_id'].replace('pair_', '')}.jpg"
        save_path = output_dir / filename
        cv2.imwrite(str(save_path), frame, [cv2.IMWRITE_JPEG_QUALITY, 95])
        
        exported_count += 1
    
    cap.release()
    return exported_count

# 建立 LabelMe 目錄
images_dir = LABELME_DIR / 'images'
annotations_dir = LABELME_DIR / 'annotations'
images_dir.mkdir(exist_ok=True)
annotations_dir.mkdir(exist_ok=True)

# 分批處理
batches = [pairs_adaptive[i:i+CHUNK_SIZE] for i in range(0, len(pairs_adaptive), CHUNK_SIZE)]
args_list = [(batch, AVI_FILE, images_dir, FLIP_RGB) for batch in batches]

print(f"\n{'='*70}")
print(f"匯出 RGB 圖片供 LabelMe 標註")
print(f"{'='*70}")
print(f"  總配對數: {len(pairs_adaptive):,}")
print(f"  翻轉 RGB: {FLIP_RGB}")
print(f"  多核處理: {NUM_WORKERS} workers")

# 多核執行
print(f"\n開始匯出...")
with ProcessPoolExecutor(max_workers=NUM_WORKERS) as executor:
    results = list(tqdm(
        executor.map(export_rgb_batch, args_list),
        total=len(batches),
        desc="匯出 RGB"
    ))

total_exported = sum(results)

print(f"\n✅ 匯出完成")
print(f"  成功匯出: {total_exported:,} 張圖片")
print(f"  位置: {images_dir}")
print("="*70)

---
## Step 11: 儲存 Thermal NPY

In [ ]:
def save_aligned_thermal_npy(pairs: List[Dict], output_dir: Path, flip: bool = False):
    """
    儲存對齊後的 Thermal 為 NPY
    """
    output_dir.mkdir(exist_ok=True)
    
    print(f"\n{'='*70}")
    print(f"儲存 Thermal 為 NPY")
    print(f"{'='*70}")
    
    for pair in tqdm(pairs, desc="儲存 Thermal"):
        thermal_img = pair['thermal']['image']  # uint8 (24, 32)
        
        if flip:
            thermal_img = cv2.flip(thermal_img, 1)
        
        save_path = output_dir / f"{pair['pair_id']}_thermal.npy"
        np.save(save_path, thermal_img)
    
    print(f"\n✅ 完成，儲存於 {output_dir}")

# 執行
THERMAL_OUT = ALIGNED_DIR / 'thermal'
save_aligned_thermal_npy(pairs_adaptive, THERMAL_OUT, FLIP_THERMAL)

---
## Step 12: 建立 LabelMe 專案環境

In [ ]:
# 類別定義
pose_classes = ['lying', 'sitting', 'fallen', 'empty', 'uncertain']

classes_file = LABELME_DIR / 'classes.txt'
with open(classes_file, 'w', encoding='utf-8') as f:
    for cls in pose_classes:
        f.write(f"{cls}\n")

print(f"✅ 類別定義已建立: {classes_file}")

# README
readme_content = f"""# LabelMe 標註專案 - 病床姿勢辨識 (v3)

## 專案資訊
- 總圖片數: {len(pairs_adaptive):,}
- 建立時間: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
- 版本: v3 - 修正時間戳配對

## ✅ 修正內容
- Thermal 時間戳手動加上毫秒偏移（125ms/幀）
- 每個 Thermal 配到不同的 RGB 幀（不再重複）
- RGB 幀間隔約 3 幀（符合 FPS 比例）

## 開始標註
```bash
cd {LABELME_DIR}
labelme images --labels classes.txt --output annotations
```

## 標註類別
- **lying**: 躺著
- **sitting**: 坐著
- **fallen**: 跌倒
- **empty**: 空床（跳過或標小區域）
- **uncertain**: 不確定

## 快捷鍵
- `Ctrl+N`: 建立新標註
- `Ctrl+S`: 儲存
- `D`: 下一張
- `A`: 上一張
"""

readme_path = LABELME_DIR / 'README.md'
with open(readme_path, 'w', encoding='utf-8') as f:
    f.write(readme_content)

print(f"\n✅ README 已建立: {readme_path}")

---
## Step 13: 最終驗證與總結

In [ ]:
# 最終驗證
print(f"\n{'='*70}")
print(f"最終驗證 - 檢查 LabelMe 圖片順序")
print(f"{'='*70}")

image_files = sorted(images_dir.glob('*.jpg'))

print(f"\n前 20 個圖片的配對結果:")
for i, img_path in enumerate(image_files[:20]):
    filename = img_path.name
    pair_id = f"pair_{filename.replace('frame_', '').replace('.jpg', '')}"
    
    matching_pair = next((p for p in pairs_adaptive if p['pair_id'] == pair_id), None)
    
    if matching_pair:
        thermal_time = matching_pair['timestamp'].strftime('%H:%M:%S.%f')[:-3]
        rgb_idx = matching_pair['rgb_frame_idx']
        
        if i > 0:
            prev_pair = next((p for p in pairs_adaptive if p['pair_id'] == f"pair_{image_files[i-1].name.replace('frame_', '').replace('.jpg', '')}"), None)
            if prev_pair:
                rgb_diff = rgb_idx - prev_pair['rgb_frame_idx']
                print(f"  {i+1:2d}. {filename:20s} → Thermal {thermal_time}, RGB 幀 {rgb_idx:5d} (RGB間隔 {rgb_diff:2d} 幀)")
            else:
                print(f"  {i+1:2d}. {filename:20s} → Thermal {thermal_time}, RGB 幀 {rgb_idx:5d}")
        else:
            print(f"  {i+1:2d}. {filename:20s} → Thermal {thermal_time}, RGB 幀 {rgb_idx:5d}")

print("\n" + "="*70)
print("資料準備完成 - 最終總結")
print("="*70)

rgb_indices = [p['rgb_frame_idx'] for p in pairs_adaptive]
intervals = [rgb_indices[i+1] - rgb_indices[i] for i in range(len(rgb_indices) - 1)]
interval_counts = Counter(intervals)

print(f"\n📊 配對統計:")
print(f"  總配對數: {len(pairs_adaptive):,}")
print(f"  成功率: {len(pairs_adaptive) / len(therm_data) * 100:.1f}%")

print(f"\n⏱️ RGB 時間精度:")
print(f"  平均誤差: {np.mean(rgb_errors):.2f}ms")
print(f"  最大誤差: {np.max(rgb_errors):.2f}ms")

print(f"\n📏 RGB 幀間隔分佈:")
for interval in sorted(interval_counts.keys())[:5]:
    count = interval_counts[interval]
    percentage = count / len(intervals) * 100
    print(f"  間隔 {interval:2d} 幀: {count:5d} 次 ({percentage:5.1f}%)")

print(f"\n✅ 檢查項目:")
interval_0_count = interval_counts.get(0, 0)
if interval_0_count == 0:
    print(f"  ✓ 無重複 RGB 幀（interval 0: {interval_0_count}）")
else:
    print(f"  ✗ 仍有重複 RGB 幀（interval 0: {interval_0_count}）")

expected_interval = int(round(FRAME_INTERVAL_MS / (1000 / fps)))
main_interval_count = interval_counts.get(expected_interval, 0)
main_interval_pct = main_interval_count / len(intervals) * 100
# Step 13 的最後部分（接續）

if main_interval_pct > 50:
    print( f"  ✓ 主要間隔符合預期（{expected_interval} 幀: {main_interval_pct:.1f}%）")
else:
    print( f"  ? 主要間隔分散（預期 {expected_interval} 幀，實際 {main_interval_pct:.1f}%）")

print(f"\n📁 輸出檔案:")
print(f"  配對分析圖: {ADAPTIVE_DIR / 'pairing_analysis_v3.png'}")
print(f"  翻轉比對圖: {DIAGNOSTIC_DIR / 'flip_comparison.png'}")
print(f"  配對 JSON: {ADAPTIVE_DIR / 'pairs_mapping_v3.json'}")
print(f"  配對 CSV: {ADAPTIVE_DIR / 'pairs_mapping_v3.csv'}")
print(f"  LabelMe 圖片: {LABELME_DIR / 'images'} ({total_exported:,} 張)")
print(f"  LabelMe README: {LABELME_DIR / 'README.md'}")
print(f"  Thermal NPY: {ALIGNED_DIR / 'thermal'}")

print(f"\n🎯 下一步:")
print(f"\n1. 檢查翻轉方向:")
print(f"   - 查看 {DIAGNOSTIC_DIR / 'flip_comparison.png'}")
print(f"   - 如需調整，修改 Step 1 的 FLIP_THERMAL 和 FLIP_RGB")
print(f"   - 重新執行 Step 10-11")

print(f"\n2. 開始 LabelMe 標註:")
print(f"   cd {LABELME_DIR}")
print(f"   labelme images --labels classes.txt --output annotations")

print(f"\n3. 標註完成後:")
print(f"   - 執行 02_convert_labelme_to_dataset.ipynb")
print(f"   - 執行 03_train_semantic_guided_sr.ipynb")

print(f"\n✨ 資料準備完成（v3 - 時間戳已修正）！")
print("="*70)

